In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Credit Card Dataset 

In [ ]:
df1=pd.read_excel('/kaggle/input/socbiz-analytics/Analytics_Project.xlsx')

In [ ]:
df=pd.DataFrame(df1)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df['PAY5'][df['PAY5']==-2].count()

There are 4546 values of outliers with -2, fixing it

The dataset has -2 value in Pay(0,1,2,3,4,5,6) columns which is wrong(since payment for CC bill before 2 months can't be done unless we know the bill amount)
Converting the -2 value to +2

In [ ]:
for (index,column) in enumerate(df):
    if (index>5 and index <12):
        for i in range (len(df[column])):
            if(df.loc[i,column]==-2):
                df.loc[i,column]=2
                #df[column].iloc[i]==2

In [ ]:
df['PAY5'][df['PAY5']==-2].count()

Since values 5 & 6 are unknown qualification therefore combining 6 to 5

Dropping 0 qualification value also

In [ ]:
df['Qualification'].value_counts()

In [ ]:
df=df[df['Qualification']!=0]

In [ ]:
df['Qualification'].value_counts()

In [ ]:
for i in range(len(df)):
    if(df['Qualification'].iloc[i]==6):
        df['Qualification'].iloc[i]=5

In [ ]:
df['Qualification'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

**EXPLORATORY DATA ANALYSIS (EDA)**

How much defaults take place?

In [ ]:
colors=sns.color_palette('flare')[2:5]
ax=df['Default'].value_counts().plot(kind="pie",figsize=(5,6),title="% Defaults",colors=colors,autopct="%1.1f%%")
plt.legend(labels=['No Default','Default'],bbox_to_anchor=(1.05,1))

In [ ]:
ax=sns.countplot(df,x="Default",palette="flare")
sns.despine(bottom=True,left=True)
#sns.color_palette("flare")
sns.set(rc={'figure.figsize':(5,4)})
sns.set_style('white')
ax.set(title="Count of Defaults of CC")
ax.set_xlabel('Default')
ax.set_ylabel('Number of Customers')
for labels in ax.containers:
    ax.bar_label(labels)

***out of 30,000 customers, 6.6k have defaulted on CC leading to a loss to the company***

***22% of CCs deafult***

**What are the factors affecting the defaults?**

How many percent of men default vs women?

In [ ]:
df['Gender'].value_counts()

In [ ]:
df['Gender'].replace({1,2},{'Male','Female'},inplace=True)
ax=sns.countplot(df[df['Default']==1],x='Gender',palette="flare")
sns.despine(bottom=True,left=True)
ax.set_xlabel('GENDER')
ax.set_ylabel('No. of Customers')
#ax.legend()
for labels in ax.containers:
    ax.bar_label(labels)

The data shows more Females default on CCs more than Male (but the percentage is different) 
* Female--3,763 of 18,112=20.7%
* Male--2,873 of 11888=24.7%

**Gender does not give enough info regarding Defaults**

How does marital status impact deafults?
How does marital status impact sepndings?

In [ ]:
df['Married'].value_counts()

In [ ]:
df['Married'].value_counts()

In [ ]:
df4=df
df4['Married']=df['Married'].replace({1,2,3},{'Married','Single','Others'})
ax=sns.countplot(df4,x="Married",hue="Default",palette="flare")
sns.despine(bottom=True,left=True)
sns.set(rc={'figure.figsize':(7,5)})
sns.set_style("white")
ax.set_ylabel("No of Customers")
plt.legend(labels=['No Default','Default'],bbox_to_anchor=(1.05,1))
for labels in ax.containers:
    ax.bar_label(labels)

In [ ]:
df4=df
df4['Married']=df['Married'].replace({1,2,3},{'Married','Single','Others'})
ax=sns.countplot(df4,x="Married",hue="Default",palette="flare")
sns.despine(bottom=True,left=True)
sns.set(rc={'figure.figsize':(7,5)})
sns.set_style("white")
ax.set_ylabel("No of Customers")
plt.legend(labels=['No Default','Default'],bbox_to_anchor=(1.05,1))

def with_hue(ax, feature, Number_of_categories, hue_categories):
    a = [p.get_height() for p in ax.patches]
    patch = [p for p in ax.patches]
    for i in range(Number_of_categories):
        total = feature.value_counts().values[i]
        for j in range(hue_categories):
            percentage = '{:.1f}%'.format(100 * a[(j*Number_of_categories + i)]/total)
            x = patch[(j*Number_of_categories + i)].get_x() + patch[(j*Number_of_categories + i)].get_width() / 2 - 0.15
            y = patch[(j*Number_of_categories + i)].get_y() + patch[(j*Number_of_categories + i)].get_height() 
            ax.annotate(percentage, (x, y), size = 12)
            
with_hue(ax,df['Married'],4,2)

***Customers with Other status of relationship have more chances of defaults.***

****How does age group impact the defaults ?****

In [ ]:
ax=sns.histplot(df,x="AGE",hue="Default",palette="flare",kde=True,binwidth=3)
sns.despine(bottom=True,left=True)
sns.set(rc={'figure.figsize':(7,5)})
sns.set_style("white")
ax.set_ylabel("No of Customers")

The above plot shows that the Age Group of 23-30 are more likely to default on payments

Also,finding out the gender of the the defaulters in age groups

In [ ]:
df2=df.loc[df['Default']==1,['AGE','Gender']]
ax=sns.histplot(df2,x="AGE",hue="Gender",palette="flare",kde=True,binwidth=3)
sns.despine(bottom=True,left=True)
sns.set(rc={'figure.figsize':(7,5)})
sns.set_style("white")
ax.set_ylabel("No of Customers")

The above plot has an interesting insight, it shows that out of all defaulters in age group of 23-30 most of them are female.

***Young females tend to default more on their CC Bill***

Which LIMIT category has most defaulters?

In [ ]:
ax=sns.histplot(df.loc[df['Default']==1],x='LIMIT_BAL',color="darkgreen",kde=True)
ax.set_ylabel('No of Customers')
sns.despine(bottom=True,left=True)
ax.set(title="Impact of CC limit on Number of Defaults")

**Customers with CC limit between (0-1lakh) are more prone default on CC payments**

Which age group lies most in the CC limit of 0-100k?

In [ ]:
df3=df.loc[df['LIMIT_BAL']<=100000,['AGE','Default']]
ax=sns.histplot(df3,x="AGE",kde=True,hue="Default",palette="flare",binwidth=3)
sns.despine(bottom=True,left=True)
ax.set_ylabel("No of Customers")

***The above plot further shows that Customers with Limit <1lakh among them, age group of 23-30 are the most defaulters.***

What is impact of Qualification on the Defaults?

In [ ]:
df['Qualification'].value_counts()

In [ ]:
df5=df
df5['Qualification'].replace({1,2,3,4,5},{'Graduate','University','High School','Other','Unknown'},inplace=True)

In [ ]:
ax=sns.countplot(df5,x="Qualification",hue="Default",palette="flare")
sns.despine(bottom=True,left=True)
sns.set(rc={'figure.figsize':(7,5)})
sns.set_style("white")
ax.set_ylabel("No of Customers")
plt.legend(labels=['No Default','Default'],bbox_to_anchor=(1.05,1))
for labels in ax.containers:
    ax.bar_label(labels)

This trend indicates that 
* 3330/14030 of graduate CC holder default on bill payments-- 23.7%
* 2036/10585 of University CC holder default--- 19.2%
* 1237/4917 Unknown qualification CC holders default--- 25.2%
* 26/331 high school qualification CC holders default---7.8%
* 7/123 Other qualification CC holders default---5.6%

***The above analysis shows that Unknown qualification CC holders 25% are likely to default 1 in every 4 customers***

***Followed by Graduate CC holders with 23.7%***

Impact of qualification
age
CC limit
CC Limit and Total amount used 
marital status vs default
marital status vs spending hue default
pay months left vs marital status and others 
threshold pay month jaha jakar default prob increases



We will be using percent to analyze the defaults

In [ ]:
#total spending in each month average of each customer
df['avg_amt']=df[['Bill1','Bill2','Bill3','Bill4','Bill5','Bill6']].mean(axis=1)

In [ ]:
df['avg_amt']

In [ ]:
df['LIMIT_BAL'].dtype

In [ ]:
df['LIMIT_BAL']=df['LIMIT_BAL'].astype('int64')

In [ ]:
df.loc[df['ID']==114,'LIMIT_BAL']

In [ ]:
ax=sns.lineplot(df,x="LIMIT_BAL",y="avg_amt",errorbar=None,hue="Default",palette='flare')
sns.despine(bottom=True,left=True)
sns.set(rc={'figure.figsize':(11,4)})
sns.set_style('white')
plt.ticklabel_format(style='plain',axis='x')
ax.set_ylabel('Average Expenses')
ax.set_xlabel('Limit of CC')

The plot shows that the average expense through CC by customers remains less than the CC limit and customers that default on bills spend less on their CC comapred to those who don't

Defaulters are customers with Limit of CC < 80k

What could be the reason for defaulters not spending more?

**At what point of percent usage of available limit do customers tend to default?
What must be the optimum % usage of CC Limit?**

In [ ]:
df6=df
df6['Percent usage']=0
df6['avg_use']=df6[['Amount1','Amount2','Amount3','Amount4','Amount5','Amount6']].mean(axis=1)
for i in range(len(df6)):
    value=(df6['avg_use'].iloc[i])*100/(df6['LIMIT_BAL'].iloc[i])
    df6.loc[i,'Percent usage']=value

In [ ]:
ax=sns.histplot(df6,x="Percent usage",hue="Default",palette="flare",kde=True)
sns.despine(bottom=True,left=True)
sns.set(rc={'figure.figsize':(5,4)})
sns.set_style('white')
ax.set_ylabel("No of Customers")

In [ ]:
df6.loc[df6['Percent usage']>100]

Customers with percent usage greater than 100% means they have delay in the Bill payments of previous months
Ignoring this we move to consider usage between 0 to 100%

In [ ]:
ax=sns.histplot(df6.loc[(df6['Percent usage']<=100) & (df6['Percent usage']>0)],x="Percent usage",hue="Default",palette="flare",kde=True)
sns.despine(bottom=True,left=True)
sns.set(rc={'figure.figsize':(5,4)})
sns.set_style('white')
ax.set_ylabel("No of Customers")

***Customers with less than 20 % usage of their CC bills per month tend to default more ***

In [ ]:
#sns.scatterplot(df6,x="",y="Percent usage",hue="Default")

***Checking if the minimum amount to be paid by the customer to sustain their CC is sufficient ?***

for every monthly expense, the bill paid is compared

bill/amount will give money given back to cc per expense by CC in case of Pay 0 for the month is not -1 (not duly paid for the month)

For August

In [ ]:
df['aug_min_pay']=0
for i in range(len(df)):
    if(df['Amount2'].iloc[i]!=0):
     value=(df['Bill1'].iloc[i])*100/(df['Amount2'].iloc[i])
    else:
        value=0
    df.loc[i,'aug_min_pay']=value

In [ ]:
df['pct_usage']=df6['Percent usage']

In [ ]:
df['aug_min_pay'].value_counts()

In [ ]:
df7=df.loc[(df['aug_min_pay']<101)&(df['aug_min_pay']>0),['aug_min_pay','Default']]
df7

In [ ]:
df8=pd.DataFrame(df7)

In [ ]:
df8

In [ ]:
ax=sns.histplot(df8,x='aug_min_pay',hue="Default")
ax.set_ylabel("No of Customers")
sns.despine(bottom=True,left=True)

The above plot shows that for August defaults, customers with payment of less than 20% of the bill of CC tend to default more than others.

Devising strategy to set a threshold payment to be done for each month's bill might reduce defaults.

Customers who paid min amount of xx percent of the total bill for August, defaulted, Instead we should look at changing the policy of minimum payment for different amount of Bill to avoid more defaults.

In [ ]:
data1=pd.read_excel('/kaggle/input/socbiz-analytics/Analytics_Project.xlsx')
data=pd.DataFrame(data1)

In [ ]:
for (index,column) in enumerate(data):
    if (index>5 and index <12):
        for i in range (len(data[column])):
            if(data.loc[i,column]==-2):
                data.loc[i,column]=2
                #df[column].iloc[i]==2

In [ ]:
data=data[data['Qualification']!=0]

# Using ML Models to predict default of a Customer's CC 

**We have to first find factors impacting the default of Credit Card payments by customers**

**We will be using correlation to find relation among various features!**

**Also, our dataset has class imbalance so we will be dealing with it.**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

In [ ]:
df['Default'].value_counts()

.values return a numpy array, so to continue with dataframe of X as train and test, we remove that

In [ ]:
x=data.iloc[:,:-1]
y=data['Default']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.3)

Using SMOTHE Algorithm to balance the dataset

In [ ]:
!pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE
smt=SMOTE(random_state=2)
x_resampled,y_resampled=smt.fit_resample(x,y)

In [ ]:
x_train_smt,x_test_smt,y_train_smt,y_test_smt=train_test_split(x_resampled,y_resampled,random_state=42,test_size=0.3)

In [ ]:
x_train_smt

Feature Selection- choosing important features from dataset which affect output most since they impact the ML model algorithm

* Reduce Overfitting
* Improve accuracy


I will be using 3 methods for feature selection:
* Univariate Selection
* Feature Importance 
* Correlation Heatmap

**Univariate Selection Method**

We have to decide which statistics to use with SelectKBest 

Since, our independent variables are numerics and numeric categories and the target variable(default) is also numeric, we will use Correlation

In [ ]:
#using scikit SelectKBest library for categorical data 

from sklearn.feature_selection import SelectKBest,chi2
from sklearn.feature_selection import f_classif

Since we have negative values in some columns that is important, we will be using other Feature selection techique --- ANOVA F-value

In [ ]:
select_feature=SelectKBest(f_classif,k=10)
select_feature.fit(x_train_smt,y_train_smt)

In [ ]:
x_train_smt.shape

In [ ]:
selected_features_df = pd.DataFrame({'Feature':list(x_train_smt.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

**The above analysis of F-test statistics using ANOVA (ANalysis Of VAriance) technique of Feature selection using SelectKBest library of Sklearn**

In [ ]:
x_train_smt_Kbest=select_feature.transform(x_train_smt)
x_test_smt_Kbest=select_feature.transform(x_test_smt)

In [ ]:
#k=10 features have been choosen for prediction 

x_train_smt_Kbest.shape

K=16 seems to be a good choice for feature selection so working again

In [ ]:
select_feature=SelectKBest(f_classif,k=16)
select_feature.fit(x_train_smt,y_train_smt)

x_train_smt_Kbest16=select_feature.transform(x_train_smt)
x_test_smt_Kbest16=select_feature.transform(x_test_smt)

In [ ]:
x_train_smt_Kbest16.shape

Heatmap for correlation

In [ ]:
corr_var=data.corr()
corr_var

In [ ]:
sns.set(rc={'figure.figsize':(24,12)})
sns.heatmap(corr_var,annot=True)

**Now, moving on to ML Model for prediction of Deafulters**

Since it is a binatry classification, we will be using following methods:
* Logistic Regression
* SVM (Support Vector Machine)
* 

**Logistic Regression Model Prediction & Analysis**

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(x_train_smt_Kbest16,y_train_smt)

In [ ]:
prediction_lr=lr.predict(x_test_smt_Kbest16)

In [ ]:
prediction_lr

**Accuracy and other metrics**

In [ ]:
accuracy_lr=accuracy_score(prediction_lr,y_test_smt)

In [ ]:
print("The accuracy of the Logistic Regression model is ", accuracy_lr*100,"%")

**Confusion matrix and TP, FN**

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [ ]:
cm_lr=confusion_matrix(y_test_smt,prediction_lr)

In [ ]:
cm_lr

The matrix shows that 

In [ ]:
import sklearn.metrics as metrics
report=metrics.classification_report(y_test_smt,prediction_lr)
print(report)

From the above Classification Report we can see various factors 

We will be discussing these:

* Precision--

**Random Forest Algorithm**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(x_train_smt_Kbest16,y_train_smt)

In [ ]:
prediction_rfc=rfc.predict(x_test_smt_Kbest16)

In [ ]:
accuracy_rfc=accuracy_score(y_test_smt,prediction_rfc)
print("The Accuracy of Random Forest Classifier is ",accuracy_rfc*100,"%")

In [ ]:
cm_rfc=confusion_matrix(y_test_smt,prediction_rfc)
cm_rfc

In [ ]:
report_rfc=metrics.classification_report(y_test_smt,prediction_rfc)
print(report_rfc)

**KNN (K-Nearest Neighbour) algorithm**

**Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb=GaussianNB()
nb.fit(x_train_smt_Kbest16,y_train_smt)

In [ ]:
prediction_nb=nb.predict(x_test_smt_Kbest16)
accuracy_nb=accuracy_score(y_test_smt,prediction_nb)
print("The Accuracy of Navie Bayes model is ",accuracy_nb*100,"%")

## CONFUSION MATRIX
print("-------------------------------")
cm_nb=confusion_matrix(y_test_smt,prediction_nb)
print(cm_nb)
print("-------------------------------")
## Classification Report
report_nb=metrics.classification_report(y_test_smt,prediction_nb)
print(report_nb)

**Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(criterion='entropy')
dtc.fit(x_train_smt_Kbest16,y_train_smt)

prediction_dtc=dtc.predict(x_test_smt_Kbest16)
accuracy_dtc=accuracy_score(y_test_smt,prediction_dtc)
print("The Accuracy of Decision Tree Classifier model is ",accuracy_dtc*100,"%")

## CONFUSION MATRIX
print("-------------------------------")
cm_dtc=confusion_matrix(y_test_smt,prediction_dtc)
print(cm_dtc)
print("-------------------------------")
## Classification Report
report_dtc=metrics.classification_report(y_test_smt,prediction_dtc)
print(report_dtc)

**XGBoost**

In [ ]:
import xgboost as xgb

In [ ]:
xgb_train=xgb.DMatrix(x_train_smt_Kbest16,y_train_smt,enable_categorical=True)
xgb_test=xgb.DMatrix(x_test_smt_Kbest16,y_test_smt,enable_categorical=True)

In [ ]:
n=50
params={
    'objective': 'binary:logistic',
    'max_depth': 3,
    'learning_rate': 0.1,
}
xg=xgb.train(params=params,dtrain=xgb_train,num_boost_round=n)

In [ ]:
prediction_xgb=xg.predict(xgb_test)
prediction_xgb=prediction_xgb.astype(int)

In [ ]:
accuracy_xgb=accuracy_score(y_test_smt,prediction_xgb)
print("The Accuracy of XGBoost model on Decision Tree is ",accuracy_xgb*100,"%")

## CONFUSION MATRIX
print("-------------------------------")
cm_xgb=confusion_matrix(y_test_smt,prediction_xgb)
print(cm_xgb)
print("-------------------------------")
## Classification Report
report_xgb=metrics.classification_report(y_test_smt,prediction_xgb)
print(report_xgb)

We will be saving the ML models that we have trained

In [ ]:
import pickle
pickle.dump(lr,open('lr_model.pkl','wb'))
pickle.dump(rfc,open('rfc_model.pkl','wb'))
pickle.dump(nb,open('nb_model.pkl','wb'))
pickle.dump(dtc,open('dct_model.pkl','wb'))
pickle.dump(xg,open('xgBoost_model.pkl','wb'))

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st

#loading the saved models

lr=pickle.load(open('/kaggle/working/lr_model.pkl','rb'))
rfc=pickle.load(open('/kaggle/working/rfc_model.pkl','rb'))
nb=pickle.load(open('/kaggle/working/nb_model.pkl','rb'))
dtc=pickle.load(open('/kaggle/working/dct_model.pkl','rb'))
xg=pickle.load(open('/kaggle/working/xgBoost_model.pkl','rb'))

In [ ]:
selected_features_df.sort_values(by='Scores', ascending=False)[:16]['Feature']

***Now, that we have analyzed some algorithms for Default prediction, we will create a simple interface for interacting with a web application to predict Default of any CC holder***

**Streamlit**

In [ ]:
%%writefile app.py
import streamlit as st
import pickle
st.set_page_config(
       page_title="Customer CC Default Prediction",
       page_icon="🧑‍💼",
       #[theme]
       #primaryColor = '#7792E3',
       #backgroundColor = '#273346',
       #secondaryBackgroundColor = '#B9F1C0',
       #textColor = '#FFFFFF',
       #font = "sans serif",
)

st.title("Customer CC Default Prediction! 🧑‍💼")
lr=pickle.load(open('/kaggle/working/lr_model.pkl','rb'))
rfc=pickle.load(open('/kaggle/working/rfc_model.pkl','rb'))
nb=pickle.load(open('/kaggle/working/nb_model.pkl','rb'))
dtc=pickle.load(open('/kaggle/working/dct_model.pkl','rb'))
xg=pickle.load(open('/kaggle/working/xgBoost_model.pkl','rb'))

def main():
    st.markdown("CC Default Predictor", unsafe_allow_html=True)
    st.markdown("Drop in The required Inputs and we will do the rest.", unsafe_allow_html=True)
    st.sidebar.header("This is a Demo web application for project of---**Analytics-SocBiz,IITR** 🧑‍💼")

#taking input buttons

name=st.text_input("Name of Customer")
age=st.number_input("Age of Customer")
Gender=st.selectbox("Gender (1- Male 2--Female)",[1,2])
Married=st.selectbox("Marital Status (1--Married 2--Single 3--Others)",[1,2,3])
Qualification=st.selectbox("Qualification 1--Graduate 2--University 3--High school 4-Other 5-Unknown",[1,2,3,4,5])
pay0=st.radio("Repayment Status in September 2005 ",(-1,2,3,4,5,6,7,8))
pay2=st.radio("Repayment Status in August 2005",(-1,2,3,4,5,6,7,8))
PAY3=st.radio("Repayment Status in July 2005",(-1,2,3,4,5,6,7,8))
PAY4=st.radio("Repayment Status in June 2005",(-1,2,3,4,5,6,7,8))
PAY5=st.radio("Repayment Status in May 2005",(-1,2,3,4,5,6,7,8))
PAY6=st.radio("Repayment Status in April 2005",(-1,2,3,4,5,6,7,8))
LIMIT_BAL=st.number_input("Limit of CC")
Bill1=st.number_input("Amount of previous payment in September 2005")
Bill2=st.number_input("Amount of previous payment in August 2005")
Bill3=st.number_input("Amount of previous payment in July 2005")
Bill4=st.number_input("Amount of previous payment in June 2005")
Bill5=st.number_input("Amount of previous payment in May 2005")
Bill6=st.number_input("Amount of previous payment in April 2005")
inputs=[[pay0,pay2,Gender,Married,PAY3,PAY4,LIMIT_BAL,PAY5,PAY6,
         Bill1,Qualification,Bill3,Bill5,Bill2,Bill6,Bill4]]

if st.button('Predict CC Default !'):
    result=lr.predict(inputs)
    updated_res = result.flatten().astype(float)
    if (updated_res==[1]):
     st.success("The LogisticRegression model predicts that Customer will : Default on the CC ")
    else:
     st.success("The LogisticRegression model predicts that Customer will : NOT Default ")

if __name__=="__main__":
    main()

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
!ngrok authtoken 2a9zDWjUmMAxFaA2tr6LcMIlnZW_7ziy7WCdhtrPkdbsvCBz3

In [ ]:
get_ipython().system = os.system

In [ ]:
!nohup streamlit run /kaggle/working/app.py &

In [ ]:
url=ngrok.connect(8080)
url

In [ ]:
!streamlit run --server.port 8080 /kaggle/working/app.py >/dev/null